# Image Pyramids

이미지 피라미드 : 서로 다른 이미지 사이즈의 set
이미지 피라미드를 사용해서 Scale을 키우면(이미지가 작아질수록 Scale은 커진다)
Sliding window가 한 번에 보는 면적이 넓어져 다른 결과가 나올 수 있다.

`Gaussian Pyramids` 와 `Laplacian Pyramids` 가 있음.

[링크](https://darkpgmr.tistory.com/137) 를 참조했다.


## Gaussian
이미지를 블러 후 홀수 번째 픽셀만 사용한다.
한 번 시행할 때마다 이미지 사이즈가 1/4로 줄어드는 특징
![가우시안](https://t1.daumcdn.net/cfile/tistory/271C783A536F3D8D24)
![가우시안2](https://t1.daumcdn.net/cfile/tistory/2140BB47536F3DE704)

In [18]:
import cv2

img = cv2.imread("Images/Sample_nizi4.jpg")

lower_reso = cv2.pyrDown(img) # 원본 이미지의 1/4 사이즈
higher_reso = cv2.pyrUp(img) # 원본 이미지의 4배

cv2.imshow('img', img)
cv2.imshow('lower', lower_reso)
cv2.imshow('higher', higher_reso)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [24]:
print("원래 이미지의 크기", img.shape)
print("축소한 이미지의 크기", lower_reso.shape)
print("확대한 이미지의 크기", higher_reso.shape)

원래 이미지의 크기 (850, 850, 3)
축소한 이미지의 크기 (425, 425, 3)
확대한 이미지의 크기 (1700, 1700, 3)


## Laplacian 피라미드
Gaussian 방법의 문제 때문에, 축소 후 확대하면 1픽셀의 차이가 나게 된다 = 외각선을 남길 수 있다.

원본 이미지 - 업스케일링한 이미지의 차이가 Laplacian 피라미드이다.

In [43]:
import cv2
img = cv2.imread("Images/Sample_nizi4.jpg")
img.shape

GAD = cv2.pyrDown(img)
print(GAD.shape)

GAU = cv2.pyrUp(GAD)
print(GAU.shape)

temp = cv2.resize(GAU, (850, 850))
res = cv2.subtract(img, temp)

cv2.imshow("res", res)
cv2.imshow("temp", temp)
cv2.waitKey(0)

cv2.destroyAllWindows()
cv2.waitKey(1)

(425, 425, 3)
(850, 850, 3)


-1

In [33]:
import numpy as np
np.array_equal(img, GAU)
# Gaussian method 의 방법 때문에, 다른 배열이 나타나게 된다.

False

이미지 Pyramid 를 이용해 이미지 결합을 자연스럽게 처리하는 법
1. 2개의 이미지를 각각 불러옴
2. 각 이미지에 대해 가우시안 피라미드 생성
3. 가우시안 피라미드를 이용해 Laplacian 피라미드 생성
4. 3을 이용하여 각 이미지의 좌측과 우측을 결함
5. 결함한 결과중 가장 작은 이미지를 확대하면서 동일 사이즈의 결합결과와 ADD하여 외각선을 선명하게 처리

In [42]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 1단계
A = cv2.imread("Images/Sample_mili1.jpg")
B = cv2.imread("Images/Sample_mili2.jpg")

# 2단계
G = A.copy()
gpA = [G]
for i in range(6):
    G = cv2.pyrDown(G)
    gpA.append(G)
    
G = B.copy()
gpB = [G]
for i in range(6):
    G = cv2.pyrDown(G)
    gpB.append(G)
    
# 3단계. Laplacian Pyramid 생성
lpA = [gpA[5]]
for i in range(5, 0 ,-1):
    GE = cv2.pyrUp(gpA[i])
    temp = cv2.resize(gpA[i-1], (GE.shape[:2][1], GE.shape[:2][0]))
    
    L = cv2.subtract(temp, GE)
    lpA.append(L)
    
lpB = [gpB[5]]
for i in range(5, 0 ,-1):
    GE = cv2.pyrUp(gpB[i])
    temp = cv2.resize(gpB[i-1], (GE.shape[:2][1], GE.shape[:2][0]))
    
    L = cv2.subtract(temp, GE)
    lpB.append(L)
    
# 4단계.
LS = []
for la, lb in zip(lpA, lpB):
    rows, cols, dpt = la.shape
    ls = np.hstack((la[:, 0:int(cols/2)], lb[:, int(cols/2):]))
    LS.append(ls)
    

# 5단계
ls_ = LS[0]
for i in range(1,6):
    ls_ = cv2.pyrUp(ls_) # upscale
    temp = cv2.resize(LS[i], (ls_.shape[:2][1], ls_.shape[:2][0]))
    ls_ = cv2.add(ls_, temp) # 선명한 이미지 생성
    
# 원본 이미지를 그대로 붙인 경우
real = np.hstack((A[:, :int(cols/2)], B[:,int(cols/2):]))

cv2.imshow('real', real)
cv2.imshow('blending', ls_)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

5
10
20
40
80
160


-1

In [39]:
rows

5